In [1]:
import pandas as pd
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import os
import face_recognition
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
from PIL import Image

In [2]:
from codes import importPath, resizeImages

# Getting all the images path:

In [3]:
folders = ['0_2', '4_6', '8_12', '15_20', '25_32', '38_43', '48_53', '60_100']

faceFiles = []
    
for files in folders:
    ff = importPath(files)
    faceFiles.append(ff)

## Create a dictionary with folder(label) / images path:

In [4]:
pics = dict()
for ffiles in faceFiles:
    for foto in ffiles:
        label = foto.split("/")[-2]
        pic = foto
        if label in pics:
            pics[label].append(pic)
        else:
            pics[label] = [pic]

## Create a dataframe with the information label / images path:

In [5]:
im = []

for k,v in pics.items():
    for a in v:
        im.append([k,a])
faces_df =  pd.DataFrame(im, columns=["label","image_path"])

display(faces_df.head())

,label,image_path
0,0_2,../OUTPUT/images/0_2/image_669.jpg
1,0_2,../OUTPUT/images/0_2/image_1570.jpg
2,0_2,../OUTPUT/images/0_2/image_346.jpg
3,0_2,../OUTPUT/images/0_2/image_441.jpg
4,0_2,../OUTPUT/images/0_2/image_1055.jpg


In [6]:
faces_df.shape

(17462, 2)

In [7]:
faces_df.label.value_counts()

25_32     5023
0_2       2491
38_43     2340
4_6       2140
8_12      2124
15_20     1642
60_100     872
48_53      830
Name: label, dtype: int64

We might need to delete some of the 25_32 data, so it doesn't interfe in the prediction results.

# Extract face from image:

Import haarcascade:

In [8]:
face_cascade = cv2.CascadeClassifier('../INPUT/haarcascade_frontalface_default.xml')

We will need to treat the images in order to detect the faces:

## Convert images into B&W:

In [9]:
def BlackandWhite(image_path):
    im_bw = cv2.imread(image_path)
    im_bw = cv2.cvtColor(im_bw, cv2.COLOR_BGR2GRAY)
    return im_bw

In [15]:
dictionary={}
count = 0
for path in faces_df['image_path']:
    dictionary[count] = BlackandWhite(path)
    count+=1

In [16]:
dictionary

{0: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 1: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 2: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 3: array([[180, 180, 181, ..., 163, 163, 162],
        [179, 180, 180, ..., 161, 161, 160],
        [178, 179, 179, ..., 161, 161, 160],
        ...,
        [ 22,  24,  26, ...,  13,  13,  13],
        [ 31,  27,  23, ...,  13,  13,  13],
        [ 34,  25,  15, ...,  13,  13,  13]], dtype=uint8),
 4: array([[

In [17]:
def configImag(array):
    try:
        faces = face_cascade.detectMultiScale(array, 1.25, 6)
        x,y,w,h = faces[0]
        face = array[y:y+h,x:x+w]
        face = cv2.resize(face,(48,48))
        return face
    except:
        return "face not found"

In [18]:
diction={}
for k,v in dictionary.items():
    diction.setdefault('clean_image', []).append(configImag(v))

In [19]:
diction

{'clean_image': [array([[ 58,  64,  76, ...,  39,  40,  43],
         [ 61,  69,  85, ...,  46,  34,  47],
         [ 67,  81,  86, ...,  45,  42,  42],
         ...,
         [193, 197, 204, ..., 142, 126, 129],
         [203, 196, 202, ..., 115, 132, 121],
         [194, 190, 191, ..., 149, 152, 146]], dtype=uint8),
  'face not found',
  array([[ 10,  13,  14, ..., 109,  85,  73],
         [ 10,  11,  11, ..., 117,  92,  73],
         [ 12,  14,  11, ..., 135,  99,  76],
         ...,
         [137, 136, 129, ..., 178, 173, 175],
         [ 92, 117, 126, ..., 162, 162, 159],
         [ 29,  25,  48, ..., 144, 144, 145]], dtype=uint8),
  'face not found',
  'face not found',
  array([[193, 194, 193, ...,  85, 159, 198],
         [194, 194, 191, ...,  86, 182, 200],
         [196, 196, 191, ..., 103, 196, 201],
         ...,
         [180, 178, 180, ..., 221, 223, 233],
         [185, 181, 180, ..., 224, 225, 237],
         [181, 177, 179, ..., 224, 219, 229]], dtype=uint8),
  array([[

In [20]:
pd.DataFrame(diction).shape

(17462, 1)

In [22]:
faces_df['arrays']= pd.DataFrame(diction)

In [24]:
faces_df[faces_df.arrays == 'face not found']

/home/carolina/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:57: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


,label,image_path,arrays
1,0_2,../OUTPUT/images/0_2/image_1570.jpg,face not found
3,0_2,../OUTPUT/images/0_2/image_441.jpg,face not found
4,0_2,../OUTPUT/images/0_2/image_1055.jpg,face not found
12,0_2,../OUTPUT/images/0_2/image_1235.jpg,face not found
16,0_2,../OUTPUT/images/0_2/image_1902.jpg,face not found
...,...,...,...
17443,60_100,../OUTPUT/images/60_100/image_636.jpg,face not found
17444,60_100,../OUTPUT/images/60_100/image_634.jpg,face not found
17445,60_100,../OUTPUT/images/60_100/image_539.jpg,face not found
17446,60_100,../OUTPUT/images/60_100/image_865.jpg,face not found


In [25]:
faces_df.drop(faces_df[faces_df.arrays == 'face not found'].index, inplace = True)

/home/carolina/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:57: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


In [29]:
print(faces_df.shape)
faces_df.head()

(13809, 3)


,label,image_path,arrays
0,0_2,../OUTPUT/images/0_2/image_669.jpg,"[[58, 64, 76, 90, 109, 119, 135, 150, 166, 164..."
2,0_2,../OUTPUT/images/0_2/image_346.jpg,"[[10, 13, 14, 22, 41, 37, 33, 35, 34, 37, 32, ..."
5,0_2,../OUTPUT/images/0_2/image_1135.jpg,"[[193, 194, 193, 188, 186, 186, 180, 146, 82, ..."
6,0_2,../OUTPUT/images/0_2/image_231.jpg,"[[72, 80, 101, 108, 110, 111, 115, 121, 128, 1..."
7,0_2,../OUTPUT/images/0_2/image_696.jpg,"[[35, 67, 100, 78, 45, 64, 75, 144, 163, 156, ..."


In [28]:
faces_df['label'].value_counts()

25_32     4202
38_43     1946
8_12      1833
4_6       1611
0_2       1536
15_20     1313
60_100     692
48_53      676
Name: label, dtype: int64

In [41]:
Faces_detected = 13809*100//17462
print(f"Faces_detected:{Faces_detected}%")

Faces_detected:79%
